In [1]:
import tabula

In [7]:
df = tabula.read_pdf("Postcode.pdf")

print (len(df))

33


In [9]:
tabula.convert_into("Postcode.pdf", "Postcode.csv", output_format="csv", pages='all')

In [8]:
import PyPDF2
# pdf file object
# you can find find the pdf file with complete code in below
pdfFileObj = open('Postcode.pdf', 'rb')
# pdf reader object
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
# number of pages in pdf
print(pdfReader.numPages)
# a page object
pageObj = pdfReader.getPage(0)
# extracting text from page.
# this will print the text you can also save that into String
print(pageObj.extractText())

1786
REGION
POSTCODE
WARD
POSTCODE
OLD POSTCODE
MTAA/VILLAGE
KITONGOJI
ILALA CBD11KIVUKONI
11101Kivukoni
Sea ViewUPANGA MASHARIKI  
11102Kitonga
Kibasila
UPANGA MAGHARIBI
11103Fire
Charambe
Mfaume
KISUTU
11104KisutuMtendeni
MCHAFUKOGE
11105Kitumbini
Mchafukoge
KARIAKOO
11106Kariakoo Magharibi
Kariakoo Kaskazini
Kariakoo Mashariki
GEREZANI
11107Gerezani Mashariki
Gerezani Magharibi
JANGWANI
11108Ukombozi
Mnazi Mmoja
Mtambani "A"
Mtambani "B"
MCHIKICHINI
11109Mission Quarter
Ilala Kota
Msimbazi Bondeni
ILALA12ILALA12101Mafuriko
Kasulu
Karume
Shariff Shamba
AmanaBUGURUNI
12102Malapa
Madenge
Kisiwani
Mvinjeni

